In [3]:
# !pip install flash_attn

In [6]:
import torch
import time
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3.5-mini-instruct", 
    device_map="cuda", 
    torch_dtype="auto", 
    trust_remote_code=True, 
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")

for name, param in model.named_parameters():
    print(f"Parameter: {name}, Type: {param.dtype}")
    break  # 첫 번째 파라미터만 출력

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Parameter: model.embed_tokens.weight, Type: torch.bfloat16


In [3]:
def get_answer_from_model(model, tokenizer, input_text, token_len):
    start_time = time.time()
    messages = [
        {"role": "system", "content": "너는 도움되는 AI 비서야. 차근 차근 생각해서 답을 알려줘"},
        {"role": "user", "content": input_text},
    ]
    
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
    )
    
    generation_args = {
        "max_new_tokens": token_len,
        "return_full_text": False,
        "temperature": 0.0,
        "do_sample": False,
    }
    
    output = pipe(messages, **generation_args)
    end_time = time.time()  # 종료 시간 기록
    execution_time = end_time - start_time  # 실행 시간 계산
    print(f"Execution Time: {execution_time:.6f} seconds")

    return output[0]['generated_text']

In [4]:
question = '한국에 어떤 새들이 존재하는지 자세히 설명해봐'
get_answer_from_model(model, tokenizer, question, 1024) # Execution Time: 40.531145 seconds

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
You are not running the flash-attention implementation, expect numerical differences.


Execution Time: 40.531145 seconds


' 한국에는 다양한 종류의 새들이 서식하고 있으며, 이들은 다양한 환경과 식물의 번성에 기여하고 있습니다. 다음은 한국의 주요 새들의 세부 정보와 그들의 특성에 대한 설명입니다:\n\n1. 노랑새 (Cettia tenebrosa): 이 새는 녹색과 갈색의 밝은 색상으로 유명합니다. 낮에는 잎으로 덮인 숲에서 생활하며, 먹이를 찾기 위해 잎을 뒤집습니다. 이 새는 번식기에 매우 소음이 많으며, 낮에는 매우 빠르게 날아다니며 번식을 위해 번식지로 이동합니다.\n\n2. 흰머리새 (Zosterops japonicus): 이 새는 흰머리새의 한 종류로, 날개와 머리에 흰색 반점이 있습니다. 이 새는 잎으로 덮인 숲과 농지에서 번식하며, 먹이를 찾기 위해 잎을 뒤집습니다. 흰머리새는 매우 빠르게 날아다니며 번식을 위해 번식지로 이동합니다.\n\n3. 노랑딱등새 (Sylvia cantillans): 이 새는 노란색과 갈색의 밝은 색상으로 유명합니다. 이 새는 잎으로 덮인 숲과 농지에서 번식하며, 먹이를 찾기 위해 잎을 뒤집습니다. 노랑딱등새는 매우 빠르게 날아다니며 번식을 위해 번식지로 이동합니다.\n\n4. 노랑말벌새 (Sylvia longicauda): 이 새는 노란색과 갈색의 밝은 색상으로 유명합니다. 이 새는 잎으로 덮인 숲과 농지에서 번식하며, 먹이를 찾기 위해 잎을 뒤집습니다. 노랑말벌새는 매우 빠르게 날아다니며 번식을 위해 번식지로 이동합니다.\n\n5. 흰머리����'

In [ ]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
import time

model_name = "microsoft/Phi-3.5-mini-instruct"  # 사용하고자 하는 모델 이름
tokenizer_name = "microsoft/Phi-3.5-mini-instruct"  # 모델에 맞는 토크나이저 이름

model = LLM(model=model_name)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

WARNING 08-22 08:44:56 arg_utils.py:776] The model has a long context length (131072). This may cause OOM errors during the initial memory profiling phase, or result in low performance due to small KV cache space. Consider setting --max-model-len to a smaller value.
INFO 08-22 08:44:56 llm_engine.py:174] Initializing an LLM engine (v0.5.4) with config: model='microsoft/Phi-3.5-mini-instruct', speculative_config=None, tokenizer='microsoft/Phi-3.5-mini-instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=Observabilit

In [ ]:
def get_answer_from_vllm(model, tokenizer, input_text, token_len):
    start_time = time.time()
    
    inputs = tokenizer(input_text, return_tensors="pt")
    
    sampling_params = SamplingParams(
        temperature=0.0,
        max_tokens=token_len,
        do_sample=False
    )
    
    output = model.generate(inputs, sampling_params=sampling_params)
    
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Execution Time: {execution_time:.6f} seconds")
    
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

In [ ]:
input_text = "한국에는 어떤 새들이 사는지 알려줘"
token_len = 1024
response = get_answer(model, tokenizer, input_text, token_len)
print(f"Generated Text: {response}")